<a href="https://colab.research.google.com/github/tkd8973/ML_Project/blob/main/ML/%EC%95%84%ED%8C%8C%ED%8A%B8_%EC%8B%A4%EA%B1%B0%EB%9E%98%EA%B0%80_2%EC%9B%94_xgboost_JW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 아파트 실거래가 예측

In [ ]:
# INPUT : 도/면적/계약년월/건축년도/유형
# OUTPUT : 금액
# EX) 서울 /50 / 2020 / 직거래 → 100억?
# EX2) 경기도/ 30 / 1990/ 중개 → ???억

# 시군구 전처리 - ex) 경기도 OO군 OO시 → 경기도 df[’’].split(’ ‘)[0]
# 거래유형 전처리 - 중개 : 0 직 : 1
# 피쳐엔지니어링 - ex) 면적당 금액?

In [ ]:
#@title 폰트 설정
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/n

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import xgboost as xgb
import seaborn as sns
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [ ]:
plt.rc('font', family='NanumBarunGothic') 

In [ ]:
#@title 아파트 실거래가 2월 데이터
df_apt2 = pd.read_csv('https://github.com/jaiwon880/ML_Projiect/raw/main/Data/%EC%95%84%ED%8C%8C%ED%8A%B8_%EB%A7%A4%EB%A7%A4__%EC%8B%A4%EA%B1%B0%EB%9E%98%EA%B0%80_20230321134120_2%EC%9B%94.csv')
df_apt2

,시군구,전용면적(㎡),계약년월,계약일,거래금액(만원),건축년도,거래유형
0,강원도 강릉시 견소동,84.9450,202302,7,"25,500",1997.0,중개거래
1,강원도 강릉시 견소동,84.9450,202302,23,"16,000",1997.0,직거래
2,강원도 강릉시 견소동,84.9450,202302,23,"23,000",1997.0,직거래
3,강원도 강릉시 견소동,84.9900,202302,11,"29,000",2005.0,중개거래
4,강원도 강릉시 교동,164.0098,202302,18,"59,500",2009.0,중개거래
...,...,...,...,...,...,...,...
32006,충청북도 충주시 호암동,84.9289,202302,16,"23,000",2007.0,중개거래
32007,충청북도 충주시 호암동,84.9289,202302,21,"23,000",2007.0,중개거래
32008,충청북도 충주시 호암동,84.6800,202302,2,"20,800",2002.0,중개거래
32009,충청북도 충주시 호암동,84.9500,202302,2,"40,000",2019.0,중개거래


In [ ]:
df_apt2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32011 entries, 0 to 32010
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시군구       32011 non-null  object 
 1   전용면적(㎡)   32011 non-null  float64
 2   계약년월      32011 non-null  int64  
 3   계약일       32011 non-null  int64  
 4   거래금액(만원)  32011 non-null  object 
 5   건축년도      32005 non-null  float64
 6   거래유형      32011 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 1.7+ MB


In [ ]:
df_apt2['거래금액(만원)'].str.strip().replace(' ','')
df_apt2

,시군구,전용면적(㎡),계약년월,계약일,거래금액(만원),건축년도,거래유형
0,강원도 강릉시 견소동,84.9450,202302,7,"25,500",1997.0,중개거래
1,강원도 강릉시 견소동,84.9450,202302,23,"16,000",1997.0,직거래
2,강원도 강릉시 견소동,84.9450,202302,23,"23,000",1997.0,직거래
3,강원도 강릉시 견소동,84.9900,202302,11,"29,000",2005.0,중개거래
4,강원도 강릉시 교동,164.0098,202302,18,"59,500",2009.0,중개거래
...,...,...,...,...,...,...,...
32006,충청북도 충주시 호암동,84.9289,202302,16,"23,000",2007.0,중개거래
32007,충청북도 충주시 호암동,84.9289,202302,21,"23,000",2007.0,중개거래
32008,충청북도 충주시 호암동,84.6800,202302,2,"20,800",2002.0,중개거래
32009,충청북도 충주시 호암동,84.9500,202302,2,"40,000",2019.0,중개거래


In [ ]:
df_apt2.describe(include='all')

,시군구,전용면적(㎡),계약년월,계약일,거래금액(만원),건축년도,거래유형
count,32011,32011.000000,32011.0,32011.000000,32011,32005.000000,32011
unique,2572,NaN,NaN,NaN,1742,NaN,2
top,인천광역시 연수구 송도동,NaN,NaN,NaN,"30,000",NaN,중개거래
freq,377,NaN,NaN,NaN,448,NaN,28974
mean,NaN,73.841925,202302.0,14.140389,NaN,2006.118419,NaN
std,NaN,22.349215,0.0,7.803201,NaN,10.559669,NaN
min,NaN,11.946000,202302.0,1.000000,NaN,1962.000000,NaN
25%,NaN,59.844400,202302.0,8.000000,NaN,1997.000000,NaN
50%,NaN,76.360000,202302.0,14.000000,NaN,2006.000000,NaN
75%,NaN,84.950000,202302.0,21.000000,NaN,2017.000000,NaN


In [ ]:
#@title 평단가 금액
df_apt2["거래금액(만원)"] = df_apt2["거래금액(만원)"].str.replace(",", "").astype(int) # 거래금액 컬럼의 쉼표를 제거하고 정수형으로 변환
df_apt2["평단가"] = (df_apt2["거래금액(만원)"] / df_apt2["전용면적(㎡)"]) * 3.3 # 평단가 계산하여 새로운 컬럼 추가
df_apt2["평단가"]

0         990.641003
1         621.578669
2         893.519336
3        1126.014825
4        1197.184558
            ...     
32006     893.688721
32007     893.688721
32008     810.581011
32009    1553.855209
32010    1620.240481
Name: 평단가, Length: 32011, dtype: float64

In [ ]:
df_apt2 = df_apt2.drop('시군구', axis=1)
df_apt2['거래유형'] = df_apt2['거래유형'].replace({'중개거래': 0, '직거래': 1})

In [ ]:
#@title 결측치 건축년도 6개 삭제 
# df_apt2.isna().sum()
df_apt2.dropna(inplace=True)

In [ ]:
df_apt2['금리'] = df_apt2.apply(lambda x: 3.5 if (x['계약년월']== 202302) else 3.25, axis=1)

In [ ]:
# 계약년월과 계약일을 합쳐서 datetime으로 변환
df_apt2['날짜'] = pd.to_datetime(df_apt2['계약년월'].astype(str) + df_apt2['계약일'].astype(str), format='%Y%m%d')

# 계약년월과 계약일 열 삭제 후 인덱스 설정
df_apt2 = df_apt2.drop(['계약년월', '계약일'], axis=1)
df_apt2 = df_apt2.set_index('날짜')

In [ ]:
df_apt2

,전용면적(㎡),계약년월,계약일,거래금액(만원),건축년도,거래유형,평단가,금리
0,84.9450,202302,7,25500,1997.0,0,990.641003,3.5
1,84.9450,202302,23,16000,1997.0,1,621.578669,3.5
2,84.9450,202302,23,23000,1997.0,1,893.519336,3.5
3,84.9900,202302,11,29000,2005.0,0,1126.014825,3.5
4,164.0098,202302,18,59500,2009.0,0,1197.184558,3.5
...,...,...,...,...,...,...,...,...
32006,84.9289,202302,16,23000,2007.0,0,893.688721,3.5
32007,84.9289,202302,21,23000,2007.0,0,893.688721,3.5
32008,84.6800,202302,2,20800,2002.0,0,810.581011,3.5
32009,84.9500,202302,2,40000,2019.0,0,1553.855209,3.5


In [ ]:
#@title 훈련셋 테스트셋 나누기

X = df_apt2.drop("거래금액(만원)", axis=1)
y = df_apt2[['거래금액(만원)']]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, random_state=100)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(25604, 7) (6401, 7) (25604, 1) (6401, 1)


In [ ]:
#@title XGBRegressor
model_xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=100)  
model_xgb.fit(X_train, y_train) 

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=100, ...)

In [ ]:
y_pred = model_xgb.predict(X_test)  # 모델 예측
mse = mean_squared_error(y_test, y_pred)  # 모델 평가
print("MSE: %.2f" % mse)

MSE: 3189874.70


In [ ]:
#@title RMSE 평가
def rmse_error(y, y_pred):
  rmse = mean_squared_error(y, y_pred) ** 0.5
  return rmse

print(rmse_error(model_xgb.predict(X_train),y_train))
print(rmse_error(model_xgb.predict(X_test),y_test))

1534.6650782728407
1786.0220310122154


In [ ]:
# 필요한 라이브러리 import
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# 데이터 로드 및 전처리
df_apt2['거래년월'] = df_apt2['계약년월'].apply(lambda x: int(str(x)[:6]))
X = df_apt2.drop(['거래금액(만원)', '계약년월', '계약일'], axis=1)  # 독립변수
y = df_apt2['거래금액(만원)']  # 종속변수
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# # 피쳐 엔지니어링
# X_train['평균매매가격'] = X_train['거래금액(만원)'] / X_train['전용면적(㎡)']
# X_test['평균매매가격'] = X_test['거래금액(만원)'] / X_test['전용면적(㎡)']

# 피쳐 셀렉션
selector = SelectKBest(f_regression, k=5)
selector.fit(X_train, y_train)
X_train = selector.transform(X_train)
X_test = selector.transform(X_test)

# 데이터 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 모델 초기화 및 학습
model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=123)
model.fit(X_train, y_train)

# 모델 평가 및 예측
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % mse)

# z-점수 계산
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X_z = (X - X_mean) / X_std
print(X_z)

MSE: 6349812.18
        전용면적(㎡)      건축년도      거래유형       평단가  금리  거래년월
0      0.496722 -0.863514 -0.323726 -0.545990 NaN   NaN
1      0.496722 -0.863514  3.088937 -0.834992 NaN   NaN
2      0.496722 -0.863514  3.088937 -0.622043 NaN   NaN
3      0.498735 -0.105914 -0.323726 -0.439982 NaN   NaN
4      4.034202  0.272886 -0.323726 -0.384251 NaN   NaN
...         ...       ...       ...       ...  ..   ...
32006  0.496001  0.083486 -0.323726 -0.621910 NaN   NaN
32007  0.496001  0.083486 -0.323726 -0.621910 NaN   NaN
32008  0.484865 -0.390014 -0.323726 -0.686990 NaN   NaN
32009  0.496945  1.219885 -0.323726 -0.104952 NaN   NaN
32010  1.161357  1.219885 -0.323726 -0.052968 NaN   NaN

[32005 rows x 6 columns]


In [ ]:
# 새로운 데이터에 대한 예측
new_data = pd.DataFrame({
    '전용면적(㎡)': [84.930],
    '계약년월': [202303],
    '계약일': [25],
    '건축년도': [2005.0],
    '거래유형': [0],
    '평단가': [971.388202],
    '금리': [3.75]
})
predicted_price = model_xgb.predict(new_data)  # 새로운 데이터에 대한 예측
print("Predicted price: %.2f" % predicted_price)  # 예측된 아파트 실거래가 출력

Predicted price: 25779.90


In [ ]:
# #@title 
# # 거래금액(만원)은 연속적인 값을 가짐 -> Regressor
# # 거래유형은 이진 분류 문제, 0 또는 1의 값으로 범주형 변수 -> Classifier 

# # from tqdm import tqdm
# # from sklearn.model_selection import TimeSeriesSplit

# # def rmse_cv(model):
# #     tscv = TimeSeriesSplit(n_splits=10)
# #     rmse_list = []
# #     model_name = model.__class__.__name__
# #     for _, (train_index, test_index) in tqdm(enumerate(tscv.split(X_train), start=1), desc=f'{model_name} Cross Validations...', total=10):
# #         X_train, X_test = X_train.iloc[train_index], X_train.iloc[test_index]
# #         y_train, y_test = y_train.iloc[train_index], y_train.iloc[test_index]
# #         clf = model.fit(X_train, y_train)
# #         pred = clf.predict(X_test)
# #         rmse = rmse_error(y_test, pred) 
# #         rmse_list.append(rmse)
# #     return model_name, rmse_list

# # def print_rmse_score(model):
# #     # cv별 프린팅, 평균 저장
# #     model_name, score = rmse_cv(model)
# #     for i, r in enumerate(score, start=1):
# #         print(f'{i} FOLDS: {model_name} RMSLE: {r:.4f}')
# #     print(f'\n{model_name} mean RMSLE: {np.mean(score):.4f}')
# #     print('='*40)
# #     return model_name, np.mean(score)

# model_xgb = xgb.XGBClassifier(random_state=100)
# model_xgb.fit(X_train, y_train)

# train_pred = model_xgb.predict(X_train)
# test_pred = model_xgb.predict(X_test)

# from sklearn.metrics import accuracy_score, confusion_matrix


# # 학습율
# print(accuracy_score(y_train, train_pred))
# print(accuracy_score(y_test, test_pred))


# model = xgb.XGBClassifier(
#     n_estimators=500,
#     max_depth=10,
#     n_jobs=-1,
#     random_state=23)
# model.fit(X_train, y_train)

# # 변수 조정 후 다시 예측
# train_pred = model.predict(X_train)
# test_pred = model.predict(X_test)

# # 학습율
# print(accuracy_score(y_train, train_pred))
# print(accuracy_score(y_test, test_pred))


# print(confusion_matrix(y_test, test_pred))
# cf_matrix = confusion_matrix(y_test, test_pred)
# cf_matrix
# group_names = ['TN','FP','FN','TP']
# group_counts = ["{0:0.0f}".format(value) for value in
#                 cf_matrix.flatten()]
# group_percentages = ["{0:.2%}".format(value) for value in
#                      cf_matrix.flatten()/np.sum(cf_matrix)]
# labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
#           zip(group_names,group_counts,group_percentages)]
# labels = np.asarray(labels).reshape(2,2)
# sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='coolwarm')
# plt.ylabel('True')
# plt.xlabel('Predicted')
# plt.show()



# # models = []
# # scores = []
# # for model in [xgb_model]:
# #     model_name, mean_score = print_rmse_score(model)
# #     models.append(model_name)
# #     scores.append(mean_score)

ValueError: ignored